In [2]:
import os
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from advertorch.context import ctx_noparamgrad_and_eval
from advertorch.test_utils import LeNet5
from advertorch_examples.utils import get_mnist_train_loader
from advertorch_examples.utils import get_mnist_test_loader
from advertorch_examples.utils import TRAINED_MODEL_PATH

In [7]:
seed = 0
mode = 'adv' # 'cln' or 'adv'
train_bs = 50 # train batch size
test_bs = 1000 # test batch size
log_interval = 200

In [8]:
torch.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if mode == 'cln':
    flag_adv_train = False
    nb_epoch = 10
    model_filename = 'mnist_lenet5_clntrained.pth'
elif mode == "adv":
    flag_advtrain = True
    nb_epoch = 90
    model_filename = "mnist_lenet5_advtrained.pth"
else:
    raise RuntimeError('mode must be "cls" or "adv"')

In [10]:
train_loader = get_mnist_train_loader(
    batch_size=train_bs, shuffle=True)
test_loader = get_mnist_test_loader(
    batch_size=test_bs, shuffle=False)

model = LeNet5()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

0.00B [00:00, ?B/s]

 99%|█████████▉| 9.80M/9.91M [00:15<00:00, 1.83MB/s]

Extracting /Users/tanimu/.advertorch/data/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz



0.00B [00:00, ?B/s]


  0%|          | 0.00/28.9k [00:00<?, ?B/s]
 57%|█████▋    | 16.4k/28.9k [00:00<00:00, 89.2kB/s]
32.8kB [00:00, 57.9kB/s]                            
0.00B [00:00, ?B/s]

Extracting /Users/tanimu/.advertorch/data/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz



  0%|          | 0.00/1.65M [00:00<?, ?B/s]
  1%|          | 16.4k/1.65M [00:00<00:21, 77.4kB/s]
  2%|▏         | 41.0k/1.65M [00:00<00:18, 89.1kB/s]
  6%|▌         | 98.3k/1.65M [00:00<00:13, 113kB/s] 
 11%|█▏        | 188k/1.65M [00:01<00:09, 149kB/s] 
 18%|█▊        | 303k/1.65M [00:01<00:06, 193kB/s]
 25%|██▍       | 410k/1.65M [00:01<00:05, 231kB/s]
 39%|███▉      | 639k/1.65M [00:01<00:03, 306kB/s]
 43%|████▎     | 713k/1.65M [00:01<00:02, 324kB/s]
 47%|████▋     | 778k/1.65M [00:02<00:02, 304kB/s]
 58%|█████▊    | 950k/1.65M [00:02<00:01, 398kB/s]
 65%|██████▍   | 1.06M/1.65M [00:02<00:01, 486kB/s]
 70%|███████   | 1.16M/1.65M [00:02<00:01, 491kB/s]
 75%|███████▌  | 1.24M/1.65M [00:02<00:00, 521kB/s]
 79%|███████▉  | 1.31M/1.65M [00:02<00:00, 532kB/s]
 85%|████████▌ | 1.41M/1.65M [00:03<00:00, 542kB/s]
 92%|█████████▏| 1.52M/1.65M [00:03<00:00, 635kB/s]
9.92MB [00:20, 496kB/s]                             [A
Exception in thread Thread-4:
Traceback (most recent call last):
  File

Extracting /Users/tanimu/.advertorch/data/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz


8.19kB [00:00, 17.7kB/s]                   

Extracting /Users/tanimu/.advertorch/data/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [12]:
if flag_advtrain:
    from advertorch.attacks import GradientSignAttack
    adversary = GradientSignAttack(
        model, loss_fn=nn.CrossEntropyLoss(reduction="sum"),
        eps=0.3, clip_min=0.0, clip_max=1.0, targeted=False)

In [14]:
for epoch in range(nb_epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ori = data
        if flag_advtrain:
            # when performing attack, the model needs to be in eval mode
            # also the parameters should be accumulating gradients
            with ctx_noparamgrad_and_eval(model):
                data = adversary.perturb(data, target)

        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(
            output, target, reduction='elementwise_mean')
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx *
                len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    model.eval()
    test_clnloss = 0
    clncorrect = 0

    if flag_advtrain:
        test_advloss = 0
        advcorrect = 0

    for clndata, target in test_loader:
        clndata, target = clndata.to(device), target.to(device)
        with torch.no_grad():
            output = model(clndata)
        test_clnloss += F.cross_entropy(
            output, target, reduction='sum').item()
        pred = output.max(1, keepdim=True)[1]
        clncorrect += pred.eq(target.view_as(pred)).sum().item()

        if flag_advtrain:
            advdata = adversary.perturb(clndata, target)
            with torch.no_grad():
                output = model(advdata)
            test_advloss += F.cross_entropy(
                output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            advcorrect += pred.eq(target.view_as(pred)).sum().item()

    test_clnloss /= len(test_loader.dataset)
    print('\nTest set: avg cln loss: {:.4f},'
          ' cln acc: {}/{} ({:.0f}%)\n'.format(
              test_clnloss, clncorrect, len(test_loader.dataset),
              100. * clncorrect / len(test_loader.dataset)))
    if flag_advtrain:
        test_advloss /= len(test_loader.dataset)
        print('Test set: avg adv loss: {:.4f},'
              ' adv acc: {}/{} ({:.0f}%)\n'.format(
                  test_advloss, advcorrect, len(test_loader.dataset),
                  100. * advcorrect / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.326441
Train Epoch: 0 [10000/60000 (17%)]	Loss: 2.177267
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.862925
Train Epoch: 0 [30000/60000 (50%)]	Loss: 1.509177
Train Epoch: 0 [40000/60000 (67%)]	Loss: 1.284981
Train Epoch: 0 [50000/60000 (83%)]	Loss: 1.360772

Test set: avg cln loss: 0.3822, cln acc: 9310/10000 (93%)

Test set: avg adv loss: 1.1265, adv acc: 6273/10000 (63%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.932174
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.784079
Train Epoch: 1 [20000/60000 (33%)]	Loss: 1.071282
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.953026
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.890544
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.599934

Test set: avg cln loss: 0.2200, cln acc: 9499/10000 (95%)

Test set: avg adv loss: 1.0391, adv acc: 6314/10000 (63%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.352931
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.723963
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.709912
Train Epoch: 2 [30000/60000

KeyboardInterrupt: 

In [ ]:
torch.save(
    model.state_dict(),
    os.path.join(TRAINED_MODEL_PATH, model_filename))